# Preparing to play

This notebook has my work towards an implementation of the play function in play.py

Initial imports

In [79]:
import pandas as pd
import numpy as np
from processing import *

Get Jeopardy data and preprocess it

In [80]:
jeopardy_data = pd.read_csv("jeopardy.csv")
preprocess(jeopardy_data)

Ensure each category has 5 or more questions, and pick 7 random categories out of those available.

In [81]:
# Makes it easier to change the variables later
categories_to_show = 7
min_questions = 5

In [82]:
categories = jeopardy_data.category
categories = categories.value_counts().apply( \
    lambda c: c >= min_questions)
categories = categories[categories == True]
categories = categories.sample(categories_to_show)
categories = categories.index
categories

Index(['COUNTING DOWN TO "MIDNIGHT"', 'DYLAN THOMAS',
       'A VISIT TO ANCIENT EGYPT', '"TEXT" ME', 'BARTENDING', 'ENDS IN "OY"',
       ''90s FICTION'],
      dtype='object')

Get 5 questions for the available categories

In [83]:
questions = jeopardy_data[jeopardy_data.category.apply( \
    lambda c: c in categories)]
category_question_count = {}
def incr(key):
    category_question_count.setdefault(key, 0)
    category_question_count[key] += 1
    return category_question_count[key] <= 5
questions = questions[questions.category.apply(incr)].reset_index(drop=True)
questions

,show_number,air_date,round,category,value,question,answer,value_float,air_year
0,3927,2001-10-02,Jeopardy!,"ENDS IN ""OY""",$100,"Yes, matey! Sailors use this word to hail a s...",ahoy,100.0,2001
1,3927,2001-10-02,Jeopardy!,"ENDS IN ""OY""",$200,"Surname that means the ""real"" thing",McCoy,200.0,2001
2,3927,2001-10-02,Jeopardy!,"ENDS IN ""OY""",$300,He's the beloved character from children's lit...,Corduroy,300.0,2001
3,3927,2001-10-02,Jeopardy!,"ENDS IN ""OY""",$400,This ancient city was also known as Ilium,Troy,400.0,2001
4,3927,2001-10-02,Jeopardy!,"ENDS IN ""OY""",$500,"Gilbert & Sullivan's ""Patience"" opened this Lo...",the Savoy,500.0,2001
5,3730,2000-11-17,Double Jeopardy!,DYLAN THOMAS,$200,"The radio play ""Under Milk Wood"" is set in Lla...",Wales,200.0,2000
6,3730,2000-11-17,Double Jeopardy!,DYLAN THOMAS,$400,Dylan may have died as a result of this activi...,Binge drinking,400.0,2000
7,3730,2000-11-17,Double Jeopardy!,DYLAN THOMAS,$600,"Thomas addressed the Blitz in ""A Refusal to Mo...",London,600.0,2000
8,3730,2000-11-17,Double Jeopardy!,DYLAN THOMAS,$800,"In ""Do Not Go Gentle Into That Good Night"", th...",His father,800.0,2000
9,3730,2000-11-17,Double Jeopardy!,DYLAN THOMAS,$1000,Dylan riffed on one of this author's titles in...,James Joyce,1000.0,2000


Print out the categories and money values for each question in the category

In [84]:
# Terminals are typically 80 characters wide
view_width = 80
cell_width, expanded_cells = divmod(view_width, categories_to_show)
cell_width -= 1  # Add space for a pipe character
cell_template_str = "{{:^{}}}|"
cell_format_str = cell_template_str.format(cell_width)

In [85]:
longest_category = max(categories, key=len)
lines, extra_line = divmod(len(longest_category), cell_width)
if extra_line > 0:
    lines += 1

for line in range(lines):
    extender = expanded_cells
    for col in categories:
        start = line * cell_width
        end = start + cell_width
        print(cell_format_str.format(col[start:end]), sep="", end="")
        if extender > 0:
            print(" ", end="")
            extender -= 1
    print()
print("-" * view_width)  # Separator between categories and monies

COUNTING D| DYLAN THOM| A VISIT TO| "TEXT" ME |BARTENDING|ENDS IN "O|'90s FICTI|
OWN TO "MI|     AS    |  ANCIENT E|           |          |    Y"    |    ON    |
 DNIGHT"  |           |    GYPT   |           |          |          |          |
--------------------------------------------------------------------------------


Print out the money values for each selected question

In [86]:
for row in range(min_questions):
    extender = expanded_cells
    for col in categories:
        money_number = questions.loc[questions.category == col].value_float.iloc[row]
        if money_number is None:
            money_number = row * 400
        money_value = questions.loc[questions.category == col].value.iloc[row]
        if money_value is None:
            money_value = "${}".format(money_number)
        print(cell_format_str.format(money_value), end="", sep="")
        if extender > 0:
            print(" ", end="")
            extender -= 1
    print("\n", "-" * view_width, sep="")

   $200   |    $200   |    $400   |    $200   |   $400   |   $100   |   $100   |
--------------------------------------------------------------------------------
   $400   |    $400   |    $800   |    $400   |   $800   |   $200   |   $200   |
--------------------------------------------------------------------------------
   $600   |    $600   |   $1200   |    $600   |  $1200   |   $300   |   $300   |
--------------------------------------------------------------------------------
   $800   |    $800   |   $1600   |   $5,200  |  $3,000  |   $400   |   $400   |
--------------------------------------------------------------------------------
  $1000   |   $1000   |   $2000   |   $1000   |  $2000   |   $500   |   $500   |
--------------------------------------------------------------------------------


Set up player's "wallet"

In [87]:
wallet = 0

Prompt for category and money value

In [88]:
quit_game = False
answers_given = 0
max_answers = categories_to_show * min_questions
def play_one_question():
    print("Your wallet: ${}".format(wallet))
    # Prompt for category
    print("Select category (1-{}, "
          "from left to right, or Q to quit)" \
          .format(categories_to_show),
          sep="", end="\n>>> ")
    category_index_choice = None
    while type(category_index_choice) != int:
        category_index_choice = input()
        if category_index_choice.lower() == "q":
            return False
        try:
            category_index_choice = int(category_index_choice)
        except ValueError:
            print("Invalid choice. Please enter a number.")
        if (category_index_choice > categories_to_show or
                category_index_choice < 1):
            print("Invalid choice.")
            category_index_choice = None
    # Prompt for question
    print("Select question (1-{}, "
          "from top to bottom, or Q to quit)" \
          .format(min_questions),
          sep="", end="\n>>> ")
    question_index_choice = None
    while type(question_index_choice) != int:
        question_index_choice = input()
        if question_index_choice.lower() == "q":
            return False
        try:
            question_index_choice = int(question_index_choice)
        except ValueError:
            print("Invalid choice. Please enter a number.")
        if (question_index_choice > min_questions or
                question_index_choice < 1):
            print("Invalid choice.")
            question_index_choice = None
    return True
while play_one_question():
    pass


Your wallet: $0
Select category (1-7, from left to right, or Q to quit)
>>> Select question (1-5, from top to bottom, or Q to quit)
>>> Your wallet: $0
Select category (1-7, from left to right, or Q to quit)
>>> 